<style>
    .renbox {
        width: 100%;
        height: 100%;
        padding: 2px;
        margin: 3px;
        border-radius: 3px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        font-weight: bolder;
        background-color: #8b8c8b;
    }
    .per {
        background-color: #ffb654;
    }
    .org {
        background-color: #49ace6;
    }
    .misc {
        background-color: #c2ccd1;
    }
    .loc {
        background-color: #3deb6c;
    }
</style>

# Partie 2 : Résolution de toponymes à l'aide d'un référentiel du Web de données

Bonjour 👋 !

Bienvenue dans la seconde partie de la séquence dédiée à l'expérimentation du **traitement automatique du langage naturel** (TALN) grâce à un **modèle d'apprentissage profond**.


## Objectifs de la séance 🎯
- expérimenter un mécanisme de liage d'entité (*Entity linking*) ;
- construire une chaîne de traitement complète de *geoparsing* dans des textes ;
- apprendre à utiliser la bibliothèque Python `requests` pour requêter l'API de la BnF et de DBPedia ;
- créer une carte de chaleur avec la bibliothèque `folium`

## Important ❗

1. Répondez aux questions directement dans les cellules de ce notebook.

2. 🆘 Une question n'est pas claire ? Vous êtes bloqué(e) ?  N'attendez pas, **appelez à l'aide 🙋**.  

3. 🤖 Vous pouvez utiliser ChatGPT/Gemini/etc. pour vous aider, **mais** contraignez vous à n'utiliser ses propositions **que si vous les comprenez vraiment**. Ne devenez pas esclave de la machine ! 🙏

4. 😌 Si vous n'avez pas réussi ou pas eu le temps de répondre à une question, **pas de panique**, le répertoire `correction/` contient une solution !

ℹ️ **Info** : La difficulté d'une question **🧩**  est indiquée de ⭐ à ⭐⭐⭐⭐.

# A/ Résolution de toponymes et géocodage avec DBPedia

## Introduction à la résolution de toponymes 🔬

La **résolution de toponymes** entre dans le domaine de l'ingénierie des connaissances en tant que cas particulier d'une tâche générale nommée **liage d'entités** (en anglais *Entity Linking - EL*).
Le liage d'entités suit l'étape de reconnaissance des entités nommées (REN/*NER*) et consiste à associer les entités annotées avec une **ressource dans une base de connaissances de référence** avec deux objectifs :

1. **désambiguïser** les entités nommées, car on considère que deux entités nommées liées à la même ressource sont en fait deux mentions de la même entité du monde réel.
2. **enrichir** les informations extraites grâce aux métadonnées que fournit la base de connaissances. Dans le cas des lieux, on sera évidemment intéressé par leurs **coordonnées géographiques**.

La résolution de toponymes est tout simplement le nom donné au liage d'entités nommées qui désignent des lieux. Si, en plus, on récupère des coordonnées géographiques pour cartographier les toponymes, la tâche devient une tâche de **géocodage** !

La base de connaissance peut être n'importe quel référentiel externe, mais les **bonnes pratiques** du domaine veulent que l'on essaye généralement de privilégier soit des **référentiels d'autorité** comme le catalogue de la BnF, soit des grands **graphes de connaissances ouverts** et publiés sur le Web de données , par exemple Wikidata ou DBPedia qui sont des versions de Wikipedia sous forme de graphes de connaissances.

Le processus de résolution de toponymes peut être décomposé en trois parties  :

<img src="./figs/schema.jpg" alt="Recherche dans un référentiel -> Désambiguisation -> Géocodage" width="80%"></img>

<span style="color: red; font-size: 1.2em;"><strong>🚨 Attention |</strong></span>  **Dans cet atelier nous négligeront l'étape de désambiguïsation.* Elle peut toutefois être essentielle selon le cas d'application; n'hésitez pas à demander d'en discuter ! 🙋


## Échauffement

Commençons par la première étape de **recherche des entités nommées lieux (LOC)**.

Pour l'exemple, dans  l'extrait de l'article sur le naufrage du titanic on trouve deux mentions de lieux, <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span> et <span class ="renbox loc">Liverpool<sup>[LIEU]</sup></span> : 


> « Le capitaine <span class ="renbox per">Smith<sup>[PERSONNE]</sup></span> qui commandait le <span class="renbox misc">Titanic<sup>[OBJET]</sup></span> est, nous l'avons dit hier, depuis plus de 35 ans au service de la <span class ="renbox org">White Star Line<sup>[ORGANISATION]</sup></span>.
> Il est actuellement agé de 60 ans.
>  Né dans le <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span>, le capitaine <span class ="renbox per">Smith<sup>[PERSONNE]</sup></span> avait fait son apprentissage de marin dans la maison d'armement <span class ="renbox org">Gibson et C°<sup>[ORGANISATION]</sup></span>, de <span class ="renbox loc">Liverpool<sup>[LIEU]</sup></span>. »


Nous allons utiliser comme référentiel de [DBPediaFR](https://fr.dbpedia.org/) (https://fr.dbpedia.org/).
DBPediaFR est une représentation en graphe de connaissances du contenu de Wikipédia en français. 

Chaque page de Wikipédia en français devient donc une **ressource RDF** dans le graphe, qui possède une **URI** et qui est enrichie de nombreuses **métadonnées**.

Par exemple, la page Wikipédia de l'École Nationale des Chartes est transformée une ressources RDF dont l'URI (qui est ici un URL) est :
> http://fr.dbpedia.org/resource/École_nationale_des_chartes

Si l'on va à cet URL dans un navigateur Web, on peut voir un résumé de toutes les triplets RDF associés à cette ressource.

Mais comment trouve-t-on l'URI d'un lieu dans DBPediaFR à partir de son nom ?
Pour cela, on peut utiliser l'outil open source  [https://fr.dbpedia.org/lookup.html](https://fr.dbpedia.org/lookup.html), qui permet justement de faire cette recherche ! 

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 1 - ⭐</strong></div>

En utilisant l'outil DBPediaFR lookup (https://fr.dbpedia.org/lookup.html), recherchez le lieu <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span>.
Rendez-vous sur la page de présentation de la meilleure resource identifiée, et recherchez parmi les triplets associés s'il y a des coordonnées géographiques. 

Quel sont les noms et les préfixes RDF des propriétés utilisées pour décrire ces informations géographiques ? Notez les dans un coin, vous en aurez besoin plus tard !

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

Vous venez de tester à la main la première étape de la  résolution de toponymes grâce à DBPedia Lookup.
Le but va maintenant être d'automatiser ce processus !


## Résoudre un toponyme avec l'API de DBPedia Lookup

[DPBedia Lookpup](https://github.com/dbpedia/dbpedia-lookup) va être très utile car son instance officielle [https://lookup.dbpedia.org/](https://fr.dbpedia.org/lookup.html) mets à disposition une API REST qui permet de l'interroger avec une simple URL.

Par exemple pour récupérer les meilleurs candidats pour le toponym *Staffordshire*, il suffit d'appeler : 


>https://fr.dbpedia.org/lookup/api/search?query=Staffordshire


In [ ]:
# Exécutez-moi ! 🚀

! curl -s "https://fr.dbpedia.org/lookup/api/search?query=Staffordshire" # -s pour le mode silencieux et ne pas afficher la progression de la requête

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Une documentation est disponible ici : https://www.dbpedia.org/resources/lookup/

Vous aurez remarqué qu'on récupère les résultats au format XML, peu pratique à manipuler en Python.
Heureusement l'outil propose un mécanisme de **négociation de contenu** qui permet de préciser un format alternatif avec le paramètre `format=`...et il serait préférable de récupérer du JSON.

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 2 - </strong></div>

Dans votre navigateur Web préféré, allez à l'URL suivante et vérifiez si les résultats renvoyés par l'API semblent correspondre à ce que vous obteniez avec la version graphique de DBPediaFR Lookup :

> https://fr.dbpedia.org/lookup/api/search?query=Staffordshire&format=JSON
 

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

## Automatisation des appels à DBPedia Lookup

Bien sûr, on souhaite automatiser ces appels à l'API afin de pouvoir résoudre des toponymes reconnus dans un texte à la volée !

Pour faire cela avec Python, nous allons utiliser la bibliothèque `requests` qui permet, comme son nom l'indique, d'exécuter des requêtes HTTP et de récupérer la réponse.

Commençons par installer `requests`

In [ ]:
# Exécutez-moi ! 🚀

%pip install -q requests

Une requête HTTP dont les paramètres sont passés via l'URL est dite de type GET.

Avec requests, on peut très facilement exécuter ce genre de requêtes et récupérer une réponse grâce à la fonction `requests.get(...)` :

In [ ]:
# Exécutez-moi ! 🚀

import requests # On importe la librairie requests

query = "https://lookup.dbpedia.org/api/search?query=Staffordshire&format=JSON" #Notre requête GET

response = requests.get(query) # On l'exécute

response.raise_for_status() # Appeler raise_for_status() après une requête GET est une bonne pratique : cela permet de lever automatiquement une exception si la requête a échoué.

print(type(response), response)

`requests.get()` renvoie donc un objet Python de type `requests.models.Response`, qui quand on l'affiche donne le code HTTP retour, ici 200, qui signifie que [tout s'est bien passé ](https://developer.mozilla.org/fr/docs/Web/HTTP/Reference/Status/200).

On peut alors récupérer le contenu de la réponse de plusieurs manières :

In [ ]:
# Exécutez-moi ! 🚀

# Sous sa forme brute :
print("Réponse 'brute':", response.content)

# Sous sa forme textuelle :
print("Réponse textuelle:", response.text)

# Sous la forme d'un JSON déja parsé et transformé en dictionnaire Python. Attention, si le contenu n'est pas du JSON, cela lèvera une exception
print("Réponse JSON:", response.json())

Dans notre cas nous sommes évidemment plutôt interessés à récupérer les ressources correspondant à une requête sous la forme d'une liste de dictionnaires Python !

Nous allons donc créer une fonction chargée de rechercher sur DBPedia un lieu passé en paramètre. 

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 3 - ⭐⭐</strong></div>

Complétez la fonction `dbpedia_lookup(toponyme: str)` ci-dessous, qui prend en paramètre un `toponyme` sous forme de chaîne de caractère, et renvoie  le contenu de l'élément racine "docs" de la réponse HTTP sous forme d'une liste de ressources représentées par des dictionnaires Python.

In [77]:
# Complétez-moi ! 🏗️

from typing import Any
from functools import cache


@cache
def dbpedia_lookup(toponyme: str) -> list[dict[str, Any]]:
    query = f"https://fr.dbpedia.org/lookup/api/search?query={toponyme}&format=JSON"
    response = ... # Complétez ici ! 🏗️
    response.raise_for_status()

    response_json = ... # Complétez ici ! 🏗️
    docs = ...  # Complétez ici ! 🏗️
    return docs or []

Vérifiez que votre fonction a le bon comportement en exécutant la cellule suivante.

In [ ]:
# Exécutez-moi ! 🚀

t = "Staffordshire"
r = dbpedia_lookup(t)

# La réponse devrait être une liste de 10 resources représentées comme des dictionnaires Python
assert all(  # Toutes les conditions suivantes doivent être vraies
    [
        isinstance(r, list),  # La réponse doit être une liste
        len(r) == 10,  # La liste doit contenir 10 éléments
        all(isinstance(x, dict) for x in r),  # Chaque élément est un dictionnaire
    ]
)

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

 <span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Chaque appel à `dbpedia_lookup()`, même avec le même toponyme, créera une requête HTTP vers l'API de DBPediaFR Lookup. Il serait très pertinent d'ajouter un mécanisme de **cache** pour s'épargner des requêtes redondantes et lentes.
 La bibliothèque standard [`functools`](https://docs.python.org/3/library/functools.html) offre une méthode de mise en cache dans la mémoire vive extrêmement simple : il suffit d'ajouter le décorateur `@cache` au dessus d'une fonction ! Ainsi, le cache conservera en mémoire chaque appel à la fonction et sa réponse : si le même appel a déjà été fait, il n'exécutera pas la fonction et renverra directement le résultat mémorisé ! 
 Par exemple :
 ```python
 from functols import cache

 @cache # Et hop, les appels à dbpedia_lookup seront mis en cache !
 def dbpedia_lookup(toponyme: str) -> list[dict[str, Any]]:
    ...



## Récupération de l'URI de la meilleure resource identifiée

DBPediaFR nous renvoie la liste des resources correspondant à la requête. C'est bien, mais ce que l'on aimerait c'est accéder aux triplets associés à cette resource afin de récupérer les coordonnées géographiques associées, s'il y en a !

Nous allons donc avoir besoin d'un mécanisme supplémentaire pour :
1. **récupérer** les URI des resources dans les dictionnaires renvoyés par `dbpedia_lookup()`.
2. **filtrer la liste** pour conserver uniquement le meilleur résultat (=le premier) pour éviter des requêtes sur des resources que l'on utilisera pas.


Nous allons pour cela créer une nouvelle fonction qui englobera `dbpedia_lookup()`, nommées `dbpedia_top1(toponyme: str)-> str | None` qui prends en paramètre un toponyme et renvoie **l'URI de la meilleure resource identifiée par DBPediaFR lookup** (ou None s'il n'y a pas de résultat).

Chaque resources JSON renvoyée par l'API de DBPediaFR Lookup contient une propriété `resource` contenant une liste d'URI pour cette resources.
Nous considérerons que l'URI de la resource est **le premier élément de cette liste**.

Par exemple, ce sera "http://dbpedia.org/resource/Staffordshire" pour le toponyme "Staffordshire" :

In [ ]:
# Exécutez-moi ! 🚀

! curl -s "https://fr.dbpedia.org/lookup/api/search?query=Staffordshire&format=JSON" | jq ".docs[0].resource" # jq est un utilitaire bash pour manipuler du JSON, très très pratique !



<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 4 - ⭐⭐</strong></div>

Complétez le corps de la fonction `dbpedia_top1(str)` dans la cellule suivante afin de récupérer **l'URI de la meilleure resources trouvées pour un toponyme passé en argument**.
Si aucun résultat n'est trouvé, la fonction doit renvoyer None

In [ ]:
# Complétez-moi ! 🏗️

from functools import cache


@cache  # On active la mise en cache pour cette fonction, cf. l'astuce précédente
def dbpedia_top1(toponyme: str) -> str | None:
    lookup = ...  # Complétez ici ! 🏗️

    if not lookup:
        return None
    else:
        return ... # Complétez ici ! 🏗️

Vérifiez que la fonction a le comportement attendu en exécutant la cellule suivante :

In [ ]:
# Exécutez-moi ! 🚀

t = "Staffordshire"
r = dbpedia_top1(t)

# La réponse doit être l'URI attendue, sous forme de chaîne de caractères
assert [isinstance(r, str), r == "http://fr.dbpedia.org/resource/Staffordshire"]

t = "Cette ressource n'existe pas "
r = dbpedia_top1(t)

# Une resource inexistante doit retourner None, pas provoquer d'erreur
assert r is None

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

## Récupération des coordonnées géographiques de la resource

Maintenant que nous savons récupérer l'URI d'une resources correspondant à un lieu, reste à récupérer ses coordonnées géographiques - en tout cas si celles-ci sont renseignées dans le graphe de DBPediaFR !

Pour cela nous allons faire utiliser un tout petit peu (promis) de SPARQL afin de requêter directement le graphe de connaissances de DBPediaFR.

Vous avez dû remarquer avec la **question 1** que les coordonnées géographiques en latitude et longitude étaient données par les propriétés `geo:lat` et `geo:long`.

Rappelez-vous, un graphe de connaissance représente l'information sous forme de triplets `<s, p, o>` où :
- `s` est le sujet du triplet, pour nous le lieu (défini par son URI);
- `p` est la propriété (ou prédicat) du triplet. Ce sont donc les propriétés `geo:lat̀` et `geo:long` qui nous intéressent;
- `o` est l'objet du triplet, ici la valeur de la coordonnée, autrement dit **ce que l'on veut récupérer** !

Voici la requête SPARQL pour récupérer les coordonnées `lat`, `long` de la ressource "Staffordshire",  identifiée par son URI DBPediaFR :

```sparql
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?lat ?long
WHERE { 
    <http://fr.dbpedia.org/resource/Staffordshire> (geo:lat) ?lat ; (geo:long) ?long . 
}
```

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 5 - </strong></div>

Exécutez cette requête dans le *SPARQL endpoint* de DBPediaFR (https://fr.dbpedia.org/sparql) et vérifiez le résultat contient bien les latitude et longitude du Staffordshire.

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Ces coordonnées sont celles d'un point, alors que Stafforshire est un comté; c'est donc une description géographique grossière d'une entité surfacique, sans doute son centroïde. Vous pouvez d'ailleurs copier ces coordonnées dans [Google Maps](https://www.google.fr/maps) pour voir quel emplacement sur Terre elles désignent.

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>


L'idée est donc d'exécuter cette requête SPARQL à la demande, avec l'URI renvoyée par `dbpedia_top1()`. 

Pour cela nous allons à nouveau utiliser `requests`, et profiter du fait qu'il est possible d'interroger le *SPARQL endpoint* et  récupérer les résultats en JSON avec une requête HTTP de type POST.

Contrairement aux requêtes GET, les requêtes POST envoient les données de la requête dans un bloc de données.
Par exemple, pour récupérer les coordonnées de "http://dbpedia.org/resource/Staffordshire" en JSON, avec l'utilitaire bash `cURL` : 

In [ ]:
# Exécutez-moi ! 🚀

! curl -s -X POST \
-H "Content-Type: application/sparql-query" \
-H "Accept: application/json" \
--data '''PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> SELECT ?lat ?long WHERE { <http://fr.dbpedia.org/resource/Staffordshire> (geo:lat) ?lat ; (geo:long) ?long. }''' \
  http://fr.dbpedia.org/sparql \
  | jq #On passe le résultat à jq pour le formater proprement

Reste donc à transformer cela en fonction Python 🙂

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Dans le résultat de la cellule précédente, on voit que les valeurs des coordonnées sont relativement "profond" dans l'objet JSON retourné, appellons le `résultat`. Pour accéder à la latitude, il faut par exemple regarder dans "results", puis "bindings", puis "lat" puis "value".  On a donc envie d'écrire `résultat["results"]["bindings"]["lat"]["value"]`. Oui mais voilà : si n'importe laquelle des clés n'existe pas, le code va planter ! On pourrait gérer ça avec des clauses imbriquées `if "clé" in résultats["..."]: ...`, mais c'est très lourd.
Mais il existe une astuce en utilisant la méthode `résultat.get("clé")` à la place de `résultat["clé"]`. Ces deux formes d'accès au contenu d'un dictionnaires sont presques similaires, sauf que :
- `résultat.get("clé")` renvoie `None` si "clé" n'est pas une clé du dictionnaire, alors que dans ce cas `résultat["clé"]` lèvera une exception ;
- `résultat.get()` accepte un second paramètre qui est la valeur retournée par défaut **si la clé n'existe pas**.

L'astuce est alors de retourner un dictionnaire vide dans ce cas, ce qui permet de chainer les appels à `résultat.get` pour parcourir les éléments imbriqués : 

In [ ]:
# Exécutez-moi ! 🚀

résultat = {"a": {"b": {"c": {"d": "valeur finale"}}}}

 # On chaîne les appels à get pour récupérer la valeur finale...
print(
    "On récupère la valeur dans a->b->c->d :",
    résultat.get("a", {}).get("b", {}).get("c", {}).get("d", None),
) 

# ... et on peut gérer les cas où la clé n'existe pas avec exactement le même code !
résultat = {"a": {"b": {}}}
print("On récupère la valeur dans a->b->c->d  mais c n'existe pas:",
    résultat.get("a", {}).get("b", {}).get("c", {}).get("d", None), # On récupère None s'il est impossible de parcourir le chemin souhaité dans le dictionnaire
)

Enfin, on peut exécuter une requête HTTP POST avec `requests.post()`, qui prend en arguments :
- l'URL à requêter
- un argument optionnel nommé `headers=` qui est un dictionnaire de *headers* HTTP;
- un argument optionnel nommé `data=` qui est donc le contenu de la requête à envoyer au serveur ;


Sachant cela et l'astuce pour parcourir un dictionnaire en main, il est temps de créer la fonction `geocode(uri: str) -> list[float]` qui :
- prends en paramètre l'URI d'une ressource "lieu" ;
- requête le *SPARQL endpoint* de DBPedia et récupère le résultat en JSON ; 
-  et renvoie ses coordonnées géographiques sous la forme d'un tuple (lat, long). Si aucune coordonnée n'existe, la fonction doit retourner une liste vide.

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 6 - ⭐⭐⭐</strong></div>

Complétez le corps de la fonction `geocode(str)` dans la cellule suivante.

In [ ]:
# Complétez-moi ! 🏗️


@cache  # Mise en cache, les requêtes SPARQL sont coûteuses en temps !
def geocode(uri: str) -> tuple[float, float]:

    # Le patron de requête SPARQL: l'URI sera substituée à la place de %s par l'argument de la fonction
    sparql_template = """PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> SELECT ?lat ?long WHERE {<%s> (geo:lat) ?lat ; (geo:long) ?long. }"""

    # On substitue l'URI dans le template de requête SPARQL en utilisant le formatage ancien de Python plutôt que les f-strings car la requête SPARQL contient déjà des accolades.
    sparql_query = sparql_template % uri

    http_headers = {
        "Content-Type": "application/sparql-query",
        "Accept": "application/json",
    }
    
    response = ... # Complétez ici  pour appeler la requête POST avec les bons arguments ! 🏗️

    response.raise_for_status()
    json_response = response.json()

    # Récupérer l'élément results -> bindings dans l'objet json_response
    # ⚠️ ATTENTION : l'élement "bindings" est une liste, donc l'argument par défaut pour get("bindings") doit être une liste vide => get("bindings", [])
    bindings = json_response.get("results", {}).get("bindings", [])

    # Si la liste "bindings" est vide, lat et long seront None
    if not bindings:
        return None, None

    # Sinon, le premier élément de la liste "bindings" est un dictionnaire contenant les valeurs de lat et long
    bindings = bindings[0]

    # Récupérer la valeur de latitude dans : lat -> value
    lat = ... # Complétez ici : utilisez l'astuce de la méthode dict.get("clé") ! Si la clé n'existe pas, lat  doit être None 🏗️

    # Récupérer la valeur de longitude dans : long -> value
    long = ... # Complétez ici comme pour lat 🏗️

    # lat et long sont récupérés en tant que chaînes de caractères, on les convertit en float
    lat = float(lat) or None
    long = float(long) or None

    return lat, long

Exécutez la cellule suivante pour vérifier sur le cas de Staffordshire que la fonction `geocode()` renvoie le résultat attendu.

In [ ]:
# Exécutez-moi !   🚀

uri = "http://fr.dbpedia.org/resource/Staffordshire"

coordonnees = geocode(uri)
print(f"{uri=} => {coordonnees=}")

assert all(
    [
        isinstance(coordonnees, tuple),
        len(coordonnees) == 2,
        all(isinstance(x, float) for x in coordonnees),
        coordonnees == (52.8333, -2.0),
    ]
)

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

## Assemblage de la chaîne de résolution de toponymes

Ne reste "plus qu'à" assembler les composants de cette petite chaîne de traitement  !

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 7 - ⭐</strong></div>

Complétez le corps de la fonction `resolution(toponyme: str) -> tuple(float,float)` qui prend en argument un toponyme, puis appelle la fonction `dbpedia_top1()` afin de récupérer la ressource de DBPedia correspondant au toponyme, et si elle existe qui appelle la fonction `geocode()` et renvoie finalement les coordonnées (lat, long) trouvées.

Note : si aucune ressource n'est trouvée, `resolution()` doit renvoyer le typle `(None, None)`.


In [ ]:
# Complétez-moi ! 🏗️


def resolution(toponyme: str) -> tuple[float, float]:

    uri = ...  # Complétez ici ! 🏗️
    
    print(f"{toponyme=} => {uri=}", end="")

    if not uri:
        lat, long = None, None
    else:
        ...  # Complétez ici ! 🏗️

    print(f" => {lat=}, {long=}")
    return lat, long

Testez la fonction avec la cellule suivante !

In [ ]:
# Exécutez-moi ! 🚀

toponyme = "Staffordshire"
assert resolution(toponyme) == (52.8333, -2.0)

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

# B/ Reconnaître automatiquement et cartographier les lieux d'un texte

<span style="color: #85d0ff; font-size: 1.2em;"><strong>ℹ️ Info |</strong></span> Cette seconde partie ne contient pas de question mais est un tutorial pour appliquer la chaîne de traitement construire à des textes dont les lieux sont reconnus à l'aide du modèle SpaCy entraîné dans la partie 1.


## Premier essai


L'idée est maintenant de pouvoir appliquer la chaîne de traitement à toutes les entités LOC extraites d'un texte historique par le modèles SpaCy entraîné dans la partie 1 !


Commençons par charger le modèle, qui devrait se trouver dans le dossier `../partie_1/ner_presse_ancienne/model-best`.

In [ ]:
# Exécutez-moi ! 🚀

import spacy

nlp = spacy.load("../partie_1/ner_presse_ancienne/model-best")

Vérifions qu'il fonctionne correctement sur le texte d'exemple `bpt6k661732w.txt` situé dans le dossier `partie_2/`.

Il s'agit du texte OCRisé par la BnF de [la une du l'Ouest Éclair du septembre 1939](https://gallica.bnf.fr/ark:/12148/bpt6k661732w/f1) (https://gallica.bnf.fr/ark:/12148/bpt6k661732w/f1) au lendemain du début de l'invasion de la Pologne par l'Allemagne nazie. On s'attend donc à y trouver mention de nombreux lieux liés aux pays impliqués.

In [ ]:
# Exécutez-moi ! 🚀

from IPython import display
from spacy import displacy

with open("bpt6k661732w.txt", "r") as file:
    texte = file.read()

doc = nlp(texte)
# ⚠️ ATTENTION, la sortie de la cellule suivante est très longue, n'hésitez pas à commenter la ligne suivante une fois que vous avez vu le résultat
# display.HTML(displacy.render(doc, style="ent", jupyter=False, page=True))

Le modèle prédit des entités de différentes classes (ORG, PER, LOC), mais nous ne sommes intéressés que par les lieux (LOC).
Filtrons donc les entités nommées reconnues pour ne ressortir que les lieux.

In [ ]:
# Exécutez-moi ! 🚀

loc = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
loc

Nous pouvons maintenant exécuter la chaîne de traitement de résolution de toponymes pour chaque entité nommée LOC !

In [ ]:
# Exécutez-moi ! 🚀

for toponyme in loc:
    resolution(toponyme)

## Géocodage d'un texte

Nous voici capables de récupérer les coordonnées des lieux dans un texte grâce à SpaCY et notre chaîne de traitement !

Afin de réutiliser le mécanisme complet, créons une fonction `geocode_texte(texte: str, model: spacy.language.Language) -> tuple[list, list]` qui :
- prend en argument un texte quelconque et le modèle SpaCy
- renvoie deux listes : une liste de toutes les mentions de lieux reconnus, et une liste des paires de coordonnées identifiées pour chacun

In [ ]:
# Exécutez-moi ! 🚀
def geocode_texte(
    text: str, nlp: spacy.language.Language
) -> tuple[list[str], list[tuple[float, float]]]:
    doc = nlp(text)
    loc = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
    coordonnees = [resolution(toponyme) for toponyme in loc]
    return loc, coordonnees

On vérifie qu'on récupère le résultat souhaité :

In [ ]:
loc, coordonnees = geocode_texte(texte, nlp)

print("Mentions de toponymes trouvées :", len(loc))
print(f"{loc=}")
print(f"{coordonnees=}")

## Cartographie avec Folium


Folium est une bibliothèque Python pour créer des cartes interactives extrêmement facilement; nous allons l'utiliser pour afficher la densité des mentions de toponymes localisés sous la forme d'une **carte de chaleur interactive** !

Commençons par installer puis importer Folium .

In [ ]:
# Exécutez-moi ! 🚀

%pip install -q folium

import folium

On peut créer une carte avec Folium en créant simplement un objet `folium.Map`

In [ ]:
# Exécutez-moi ! 🚀

folium_map = folium.Map(
    location=[48.8566, 2.3522],  # On précise les coordonnées du centre de la carte...
    zoom_start=3,  # ... et le niveau de zoom initial. 0 = vue du monde, 18 = vue très rapprochée
)

# On affiche l'objet Map
folium_map

Ajouter des couches cartographiques est également très facile.

Pour créer une carte de chaleur, il suffit de créer un objet de type `HeatMap`, disponible dans le module `folium.plugins`.

L'objectif est donc de créer une carte de chaleur à partir des lieux géocodées dans un texte grâce à  `geocode_texte()`.

Le constructeur de `HeatMap` accepte une liste de coordonnées géographiques **valides**. Nous devons donc préalablement retirer les paires de coordnnées `(None, None)` potentiellement produites par la chaîne de traitement lorsque des toponymes ne sont pas résolus.


In [ ]:
# Exécutez-moi ! 🚀

coordonnees_valides = [(lat, long) for lat, long in coordonnees if lat and long]

print("Coordonnées valides :", len(coordonnees_valides), "/", len(coordonnees))

Il ne reste plus qu'à créer un objet `HeatMap` à partir de ces coordonnées valides, puis l'ajouter à la carte !

In [ ]:
# Exécutez-moi ! 🚀

from folium.plugins import HeatMap

# On crée une carte de chaleur à partir des coordonnées valides
heatmap = HeatMap(coordonnees_valides)

# On ajoute la carte de chaleur à la carte folium
heatmap.add_to(folium_map)

Ce qui permet de construire la projection cartographique finale des mentions de lieux identifiés dans le texte de presse :

In [ ]:
# Exécutez-moi ! 🚀

folium_map

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

# Ouf, c'est fini ! 🏁

C'est tout pour cette fois, vous voici arrivé(e)s au bout, félicitations ! 🎉🎉

Si vous voulez expérimenter d'avantage, **lancez l'application Dash dashboard.py** qui réutilise le code créé ici et permet de projeter sur une carte un texte récupéré depuis Gallica grâce à son [API Document](https://api.bnf.fr/fr/api-document-de-gallica) !  

In [ ]:
# Exécutez-moi ! 🚀

! python ./dashboad.py